In [1]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import matplotlib.pyplot as plt
from nltk import tokenize
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/xiaoxiao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#import data
import pandas as pd
import os
import csv
import re
from translate import Translator
import spacy
import langid
cwd = os.getcwd()
daen = pd.read_csv('11_countries.csv')

In [6]:
# change the column orders
mydata = daen[['screen_name','country_code','country_name','tweet','favourites_count','followers_count','statuses_count','friends_count','listed_count','bounding_box','image_url','banner_url','centroid']]
mydata.head()

,screen_name,country_code,country_name,tweet,favourites_count,followers_count,statuses_count,friends_count,listed_count,bounding_box,image_url,banner_url,centroid
0,bangsadpeople,ID,Indonesia,"Skin trial doang belagu sia,\nDah ah cape mo s...",3202.0,636.0,4754.0,442.0,1.0,"[[[107.666224, -6.938641], [107.666224, -6.904...",https://pbs.twimg.com/profile_images/130582291...,https://pbs.twimg.com/profile_banners/10518289...,"(-6.921403, 107.67888099999999)"
1,HugsOfPoetry,BR,Brazil,@Pitty É como você já dizia na letra genial de...,9110.0,171.0,5648.0,597.0,0.0,"[[[-48.285982, -16.052405], [-48.285982, -15.5...",https://pbs.twimg.com/profile_images/128967808...,https://pbs.twimg.com/profile_banners/10595783...,"(-15.776254, -47.796623000000004)"
2,papillonhero,TR,Turkey,@ruhsuzkarbonat Yapacak bişi yok...daha iyilerine,1937.0,236.0,2555.0,601.0,0.0,"[[[28.632104, 40.802734], [28.632104, 41.23990...",https://pbs.twimg.com/profile_images/127750686...,https://pbs.twimg.com/profile_banners/78716328...,"(41.0213205, 29.005222500000002)"
3,abhi_kumar_pth,IN,India,"""Samay"" aur ""shabd"" dono ka upayog laparvahee ...",279.0,7.0,56.0,23.0,0.0,"[[[86.054331, 19.850414], [86.054331, 19.90745...",https://pbs.twimg.com/profile_images/125880954...,https://pbs.twimg.com/profile_banners/12575068...,"(19.878935, 86.08442550000001)"
4,M30000o,SA,Saudi Arabia,ان الله لا يخيب عبدا تفائل به خيرا💙 https://t....,2.0,5337.0,696.0,218.0,3.0,"[[[46.304398, 24.332068], [46.304398, 25.11181...",https://pbs.twimg.com/profile_images/119581213...,https://pbs.twimg.com/profile_banners/11958119...,"(24.721943500000002, 46.685077)"


In [7]:
#collect the tweet variable
no_url_data= pd.DataFrame(mydata.tweet) 
print(no_url_data)

                                                    tweet
0       Skin trial doang belagu sia,\nDah ah cape mo s...
1       @Pitty É como você já dizia na letra genial de...
2       @ruhsuzkarbonat Yapacak bişi yok...daha iyilerine
3       "Samay" aur "shabd" dono ka upayog laparvahee ...
4       ان الله لا يخيب عبدا تفائل به خيرا💙 https://t....
5                  @ReceinAja Kek ada manis manisnya gitu
6       Que deus abençoe minha família amém....\n#BoaT...
7       Parece que ninguém se importa, o que aconteceu...
8       who tryna do that mamma mia 2 scene ill fr buy...
9                               segura a mão de qm te ama
10      #USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...
11                          E atô até agora na Rocinha...
12            Alexander’s on my feet but ima king doe ! 🤣
13                                https://t.co/ONEYjodpYc
14      https://t.co/rjRpMj9Py0 Rocks Off Plug Petite ...
15                    this still Denzels best performance
16            

In [8]:
#delete url 
no_url=no_url_data['tweet'].str.replace('http\S+|www.\S+', '', case=False)
print(no_url)

0         Skin trial doang belagu sia,\nDah ah cape mo s...
1         @Pitty É como você já dizia na letra genial de...
2         @ruhsuzkarbonat Yapacak bişi yok...daha iyilerine
3         "Samay" aur "shabd" dono ka upayog laparvahee ...
4                      ان الله لا يخيب عبدا تفائل به خيرا💙 
5                    @ReceinAja Kek ada manis manisnya gitu
6         Que deus abençoe minha família amém....\n#BoaT...
7         Parece que ninguém se importa, o que aconteceu...
8         who tryna do that mamma mia 2 scene ill fr buy...
9                                 segura a mão de qm te ama
10        #USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...
11                            E atô até agora na Rocinha...
12              Alexander’s on my feet but ima king doe ! 🤣
13                                                         
14         Rocks Off Plug Petite Sensations Anal Plug N1...
15                      this still Denzels best performance
16                                      

In [9]:
#remove punctuation characters except hashtag
no_punc = no_url.str.replace('[^\w\s#]', '', flags=re.UNICODE)
print(no_punc)

0         Skin trial doang belagu sia\nDah ah cape mo sl...
1         Pitty É como você já dizia na letra genial de ...
2             ruhsuzkarbonat Yapacak bişi yokdaha iyilerine
3         Samay aur shabd dono ka upayog laparvahee se n...
4                       ان الله لا يخيب عبدا تفائل به خيرا 
5                     ReceinAja Kek ada manis manisnya gitu
6           Que deus abençoe minha família amém\n#BoaTarde 
7         Parece que ninguém se importa o que aconteceu ...
8         who tryna do that mamma mia 2 scene ill fr buy...
9                                 segura a mão de qm te ama
10        #USGS07343000  N Sulphur Rv nr Cooper TX \nHea...
11                               E atô até agora na Rocinha
12                 Alexanders on my feet but ima king doe  
13                                                         
14         Rocks Off Plug Petite Sensations Anal Plug N1...
15                      this still Denzels best performance
16                                      

In [10]:
mydata['tweets']=no_punc
mydata.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,screen_name,country_code,country_name,tweet,favourites_count,followers_count,statuses_count,friends_count,listed_count,bounding_box,image_url,banner_url,centroid,tweets
0,bangsadpeople,ID,Indonesia,"Skin trial doang belagu sia,\nDah ah cape mo s...",3202.0,636.0,4754.0,442.0,1.0,"[[[107.666224, -6.938641], [107.666224, -6.904...",https://pbs.twimg.com/profile_images/130582291...,https://pbs.twimg.com/profile_banners/10518289...,"(-6.921403, 107.67888099999999)",Skin trial doang belagu sia\nDah ah cape mo sl...
1,HugsOfPoetry,BR,Brazil,@Pitty É como você já dizia na letra genial de...,9110.0,171.0,5648.0,597.0,0.0,"[[[-48.285982, -16.052405], [-48.285982, -15.5...",https://pbs.twimg.com/profile_images/128967808...,https://pbs.twimg.com/profile_banners/10595783...,"(-15.776254, -47.796623000000004)",Pitty É como você já dizia na letra genial de ...
2,papillonhero,TR,Turkey,@ruhsuzkarbonat Yapacak bişi yok...daha iyilerine,1937.0,236.0,2555.0,601.0,0.0,"[[[28.632104, 40.802734], [28.632104, 41.23990...",https://pbs.twimg.com/profile_images/127750686...,https://pbs.twimg.com/profile_banners/78716328...,"(41.0213205, 29.005222500000002)",ruhsuzkarbonat Yapacak bişi yokdaha iyilerine
3,abhi_kumar_pth,IN,India,"""Samay"" aur ""shabd"" dono ka upayog laparvahee ...",279.0,7.0,56.0,23.0,0.0,"[[[86.054331, 19.850414], [86.054331, 19.90745...",https://pbs.twimg.com/profile_images/125880954...,https://pbs.twimg.com/profile_banners/12575068...,"(19.878935, 86.08442550000001)",Samay aur shabd dono ka upayog laparvahee se n...
4,M30000o,SA,Saudi Arabia,ان الله لا يخيب عبدا تفائل به خيرا💙 https://t....,2.0,5337.0,696.0,218.0,3.0,"[[[46.304398, 24.332068], [46.304398, 25.11181...",https://pbs.twimg.com/profile_images/119581213...,https://pbs.twimg.com/profile_banners/11958119...,"(24.721943500000002, 46.685077)",ان الله لا يخيب عبدا تفائل به خيرا


In [11]:
final_data = mydata[['screen_name','country_code','country_name','tweets']]
final_data

,screen_name,country_code,country_name,tweets
0,bangsadpeople,ID,Indonesia,Skin trial doang belagu sia\nDah ah cape mo sl...
1,HugsOfPoetry,BR,Brazil,Pitty É como você já dizia na letra genial de ...
2,papillonhero,TR,Turkey,ruhsuzkarbonat Yapacak bişi yokdaha iyilerine
3,abhi_kumar_pth,IN,India,Samay aur shabd dono ka upayog laparvahee se n...
4,M30000o,SA,Saudi Arabia,ان الله لا يخيب عبدا تفائل به خيرا
5,kecapbangauuwuu,ID,Indonesia,ReceinAja Kek ada manis manisnya gitu
6,FloresbelaV,BR,Brazil,Que deus abençoe minha família amém\n#BoaTarde
7,aleemsiilva,BR,Brazil,Parece que ninguém se importa o que aconteceu ...
8,fuhjeeta,US,United States,who tryna do that mamma mia 2 scene ill fr buy...
9,lucxmauricio,BR,Brazil,segura a mão de qm te ama


In [22]:
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(final_data['tweets'].shape[0]))

# Display 10 random rows from the data.
tweets=final_data['tweets'].sample(100)
tweets

Number of training sentences: 402,612



199158    クール女子冷静で落ち着きがある人何があっても取り乱さないクールビューティーだけど恋人には甘え...
382716    посмотрела промо шестой серии и у меня как и у...
52883     COYB Not paying to stream againsaving money fo...
160377                           miraccumbur Ne eksik kanka
1447      SnakeBadAss Right Guy sitting in the waiting r...
358427    DonaNetas MemeYamelCA Ya sabes cómo es la meme...
145210                                          chan_kshow 
387806             Acabou de publicar uma foto em Lichinga 
115056                                      Quero ir embora
205073            NOTHING MAKES ME ANGRIER THAN GRAZE BOXES
258286    Kimseyi merak etmiyorum kimsenin başarısını kı...
305000                              busrakllcc İyi geceler 
159273    knp ya abis makan lse selalu pengen muntah tp ...
144487    Blessed Mabon equinox to all I hope the new se...
370818        شكرا للأيام الي وضحت لنا بأن الكلام يبقى كلام
112905               Mell_Nogueira1 Son otras cabezas amiga
260489                                  

In [23]:
# find the max length sentences
max_len = 0

# For every sentence...
for twt in tweets:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(twt, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  63


In [28]:
# tokenized the sentences with token ids
MMAX_LEN = 64

sentences = tweets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    for row in data:
        sents = tokenize.sent_tokenize(row)
    
    for sent in sents:
        encoded_sent = tokenizer.encode_plus(text=sent,
                                          add_special_tokens=True,
                                          max_length=MAX_LEN,
                                          pad_to_max_length=True,
                                          return_attention_mask=True,
                                          truncation=True,
                                          return_tensors = 'pt',     # Return pytorch tensors.
                                           )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
  # Convert lists to tensors
    input_ids = torch.cat(input_ids)
    attention_masks = torch.cat(attention_masks)

    return input_ids, attention_masks


tokens_tensor, segments_tensor = preprocessing_for_bert(sentences)

# Print sentences, now as a list of IDs.
print('Original: ', sentences)
print('Token IDs:', input_ids)



Token IDs: [101, 124, 4348, 8137, 1946, 111831, 111820, 37407, 4787, 7695, 2226, 1940, 7536, 7394, 2039, 47155, 90610, 7860, 2316, 1910, 5660, 1976, 4704, 1938, 16923, 6670, 1981, 22946, 2179, 1940, 6209, 19945, 8140, 28858, 15802, 3644, 103369, 16923, 11937, 50508, 52310, 18767, 50508, 38352, 106975, 25204, 8406, 1906, 14803, 53106, 102]


/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [29]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [33]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensor)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
    print(hidden_states)

(tensor([[[-0.3829,  0.1298, -0.1524,  ...,  0.2827, -0.4924, -0.0289],
         [ 0.0676,  0.6397, -0.1611,  ...,  0.5606, -0.4208, -1.1830],
         [ 0.8210,  0.2030,  1.4503,  ...,  0.2733, -0.6050,  0.3479],
         ...,
         [ 0.7518,  0.1399, -0.5572,  ...,  0.9775,  0.9232,  0.3175],
         [ 0.6375,  0.1817, -0.2745,  ...,  0.8228,  0.9863,  0.6107],
         [ 0.7481, -0.5418, -0.1858,  ...,  0.8308,  0.6889,  0.4052]]]), tensor([[[-0.1175,  0.0661, -0.0268,  ...,  0.1421, -0.1443,  0.0997],
         [ 0.0546,  0.4906,  0.3629,  ...,  0.4896, -0.0325, -0.9445],
         [ 0.7082,  0.4606,  1.6036,  ...,  0.3104, -0.2436,  0.2974],
         ...,
         [ 0.3667,  0.5452, -0.3926,  ..., -0.0661,  0.2936,  0.3326],
         [ 0.3080,  0.5971, -0.2313,  ..., -0.1026,  0.3692,  0.5012],
         [ 0.4256,  0.0739, -0.1338,  ..., -0.0622,  0.1066,  0.3880]]]), tensor([[[-0.0733, -0.0761, -0.0198,  ...,  0.0649, -0.0550,  0.0472],
         [-0.2552,  0.2341,  0.3869,  ...,

In [32]:
token_embeddings = torch.stack(hidden_states, dim=0)
# Average over batches
token_embeddings = token_embeddings.mean(1)
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()


torch.Size([64, 13, 768])

In [34]:
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 64, 768])